In [10]:
import librosa
from typing import List, Union, Optional, Tuple, Set
import re
import os
import glob
import numpy as np
import sys

In [1]:
# the main audio class will create audio objects that will store numpy arrays and sample rates of one or more audio files
# the user can:
    # specify the path to one file
    # specify the path to a folder containing multiple sound files
    # specify a list of paths to multiple individual files or folders containing files
        # everything in a folder will be included by default
        # use a regex string to select specific files in a folder

    # provide a list of files to ignore (none by default)
    # provide a list of folder paths to ignore (everything ignored by default - overrides include)
    # use regex to specify particular name patterns to ignore

class Audio:
    
    def __init__(self, include: Union[str, Tuple[str, ...]], exclude: Optional[Union[str, Tuple[str, ...]]] = None, recursive: bool = False) -> None:
        self.samples = {}
        self.fetch(include, exclude, recursive)
    
    def fetch(self, include: Union[str, Tuple[str, ...]], exclude: Optional[Union[str, Tuple[str, ...]]] = None, recursive: bool = False, append=False) -> bool:
        """
        Fetch files from specified directories with include and exclude patterns.

        :param include: A string or tuple of strings specifying the directories or patterns to include.
        :param exclude: A string or set of strings specifying the patterns to exclude.
        :param recursive: Boolean indicating whether to search directories recursively.
        :return: List of file paths that match the include patterns and do not match the exclude patterns.
        """
        if isinstance(include, str):
            include = (include,)
            
        if exclude is None:
            exclude = ()
        elif isinstance(exclude, str):
            exclude = (exclude,)

        audio_files = []
        
        for path in include:
            if os.path.isdir(path):
                audio_files.extend(self._fetch_from_directory(path, exclude, recursive))
            elif os.path.isfile(path):
                if not self._is_excluded(path, exclude):
                    audio_files.append(path)
            else:
                print(f"{path} does not exist or is not a regular file/directory.")
        
        # reseting the sample dictionary with freshly loaded data
        if(not append):
            self.samples.clear()
            
        for f in self.fetch(include, exclude, recursive):
            if(append and f in self.samples.keys()):
                continue
            try:
                sample, sr = librosa.load(f)
                self.samples[f] = (sample, sr)
            except Exception as e:
                print(f"Error loading {f}: {e}")
                return False
        
        return True
    
    def _fetch_from_directory(self, directory: str, exclude: Tuple[str, ...], recursive: bool) -> List[str]:
        """
        Fetch files from a directory based on include and exclude patterns.

        :param directory: The directory to search.
        :param exclude: Set of patterns to exclude.
        :param recursive: Boolean indicating whether to search directories recursively.
        :return: List of file paths.
        """
        directory = directory if directory.endswith('/') else (directory + '/')
        patterns = ["*.wav", "*.mp3", "*.flac", "*.ogg", "*.m4a", "*.aac", "*.wma", "*.aiff", "*.au", "*.amr",]  # Add your patterns here
        files = []
        
        for pattern in patterns:
            if recursive:
                files.extend(glob.glob(directory + '**/' + pattern, recursive=True))
            else:
                files.extend(glob.glob(directory + pattern))
        
        return [f for f in files if not self._is_excluded(f, exclude)]
    
    def _is_excluded(self, file_path: str, exclude: Tuple[str, ...]) -> bool:
        """
        Check if a file path matches any of the exclude patterns.

        :param file_path: The file path to check.
        :param exclude: Set of patterns to exclude.
        :return: Boolean indicating whether the file path is excluded.
        """
        # use regex or regular wildcard expressions for better exclusion matching
        for e in exclude:
            pattern = re.compile(e)
            if(pattern.search(file_path)):
                return True
        return False

NameError: name 'Union' is not defined

In [6]:
audio = Audio("./RAVDESS/Actor_14", "^\d{2}-\d{2}")

False